Scraper node:

In [1]:
#scraper node
from nespaper3k_nodes import newspaper3k_scraper
scraper = newspaper3k_scraper()

#you can also provide a header for the request and a timeout for the page loading
scraper = newspaper3k_scraper(
    headers={'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'},
    request_timeout= 10)

/Users/josepsmachine/miniforge3/envs/haystacky/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#to run in standalone use run.
scraper.run?

Signature:
scraper.run(
    query: str,
    lang: str = None,
    metadata: bool = False,
    links: bool = False,
    keywords: bool = False,
    summary: bool = False,
    path: str = None,
    load: bool = False,
    verbose_fails: bool = True,
)
Docstring:
:param query: String containing the webpage to scrape.
:param lang: (None by default) language to process the article with, if None autodetected.
    Available languages are: (more info at https://newspaper.readthedocs.io/en/latest/)
    input code      full name

    ar              Arabic
    ru              Russian
    nl              Dutch
    de              German
    en              English
    es              Spanish
    fr              French
    he              Hebrew
    it              Italian
    ko              Korean
    no              Norwegian
    fa              Persian
    pl              Polish
    pt              Portuguese
    sv              Swedish
    hu              Hungarian
    fi              Finnish

In [3]:
# You can also use run_batch and provide a set of start urls
scraper.run_batch?

Signature:
scraper.run_batch(
    query: list,
    lang: str = None,
    metadata: bool = False,
    links: bool = False,
    keywords: bool = False,
    summary: bool = False,
    path: str = None,
    load: bool = False,
    verbose_fails: bool = True,
)
Docstring:
:param query: list of strings containing the webpages to scrape.
:param lang: (None by default) language to process the article with, if None autodetected.
    Available languages are: (more info at https://newspaper.readthedocs.io/en/latest/)
    input code      full name

    ar              Arabic
    ru              Russian
    nl              Dutch
    de              German
    en              English
    es              Spanish
    fr              French
    he              Hebrew
    it              Italian
    ko              Korean
    no              Norwegian
    fa              Persian
    pl              Polish
    pt              Portuguese
    sv              Swedish
    hu              Hungarian
    fi    

In [4]:
#you can save scraped articles's html in a folder like this:
scraper.run(query="articles",
    metadata=True,
    summary=True,
    keywords=True,
    load=True,
    verbose_fails=False)

Scraping: articles/https:__www.emb-norw...:  87%|████████▋ | 224/258 [01:22<00:45,  1.33s/it]

In [12]:
#can also scrape a local html file by having the query be the path of the file and load=True
scraper.run(query="articles/https:__www.lonelyplanet.com_articles_getting-around-norway.html",
    metadata=True,
    summary=True,
    keywords=True,
    load=True)

({'documents': [<Document: {'content': 'With glacier-hewn fjords and deep green forests to gaze upon, traveling around Norway is an arresting experience however you choose to do it.\n\nServed by a combination of state-wide, private and municipal companies, Norway’s transportation network is extensive and efficient. And although journeys can be long, services across the board are usually very comfortable. You can buy tickets for almost every trip online or via an app, often at a discount. A good place to start is the national public transport planner EnTur (also available as an app). It pulls together routes on all modes of transport in the country, though you’ll mostly have to purchase tickets directly from each service operator.\n\nScout new ways to explore the planet\'s wildest places with our weekly newsletter delivered to your inbox.\n\nBoat is the most scenic way to travel\n\nFerries crossing the fjords take cars, foot passengers and cyclists – simply pay the attendant on board. T

In [13]:
#example of using the scrape node in a pipeline.

from qdrant_haystack.document_stores import QdrantDocumentStore
from haystack.nodes import EntityExtractor
from haystack.pipelines import Pipeline
from haystack.nodes import PreProcessor

document_store = QdrantDocumentStore(
    ":memory:",
    index="Document",
    embedding_dim=768,
    recreate_index=True,
)

entity_extractor = EntityExtractor(model_name_or_path="dslim/bert-base-NER",flatten_entities_in_meta_data=True)

processor = PreProcessor(
    clean_empty_lines=False,
    clean_whitespace=False,
    clean_header_footer=False,
    split_by="sentence",
    split_length=30,
    split_respect_sentence_boundary=False,
    split_overlap=0
)

indexing_pipeline = Pipeline()
indexing_pipeline.add_node(component=scraper, name="scraper", inputs=['File'])
indexing_pipeline.add_node(component=processor, name="processor", inputs=['scraper'])
indexing_pipeline.add_node(entity_extractor, "EntityExtractor", ["processor"])
indexing_pipeline.add_node(component=document_store, name="document_store", inputs=['EntityExtractor'])

#we can pass the previously seen arguments also
indexing_pipeline.run(query = "https://www.roughguides.com/norway/",
    params={
        "scraper":{
            "metadata":True,
            "summary":True,
            "keywords":True
        }
    })

Extracting entities: 100%|██████████| 2/2 [00:01<00:00,  1.28it/s]
10000it [00:00, 1241616.29it/s]      


{'documents': [<Document: {'content': 'Norway stretches north in a long, slender band from the Skagerrak, the choppy channel that separates the country from Denmark, its coastline battered and buffeted by the Atlantic as it juts up towards the Arctic Sea. Behind this rough and rocky coast are spectacular mountain ranges, harsh upland plateaux, plunging river valleys, rippling glaciers, deep forests and mighty fjords of unsurmounted beauty – an exhilarating landscape begging to be explored by car, boat or bike, on skis or even husky-drawn sled. Perhaps inevitably, the fjords are the apple of the tourist industry’s eye – with the infrastructure to prove it – though when well-heeled English and German gentlemen travellers arrived here in the late nineteenth century on the hunt for the Scandinavian exotic, Norwegians were so poor that you could hire a gillie or two for next to nothing. It is this stark contrast – between a severely impoverished past and an astoundingly wealthy present – th

crawler node:

In [14]:
#The crawler node uses the scraper node to create documents from articles.
from nespaper3k_nodes import newspaper3k_crawler

crawler = newspaper3k_crawler(
    headers={'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'},
    request_timeout= 10)

In [15]:
#the main parameters to take into account are n_articles and beam. 
#n_articles will limit to how many articles to scrape.
#The beam value will determine how many articles from a webpage should be scraped before 
#continuing to scrape webpages found in "deeper" webpages. 
#0 will result in Breadth first search, exhausting all links to visit per page and 
#1 will only follow (theorically until no dead end is reached) 1 link per scraped article (basically performing Depth first search)
#>1 any number greater than one would be performing some type of "bread first beam search".
crawler.run?

Signature:
crawler.run(
    query: str,
    n_articles: int,
    beam: int = 0,
    filters: list = None,
    keep_links: bool = False,
    lang: str = None,
    metadata: bool = False,
    links: bool = False,
    keywords: bool = False,
    summary: bool = False,
    path: str = None,
)
Docstring:
:param query: initial url to start scraping
:param n_articles: number of articles to scrape
:param beam: number of articles from each scraped website to prioritize in the crawl queue.
    If 0 then priority of scrape will be a simple continuous pile of found links after each scrape. (BFS). 
    If 1 then would be performing (DFS).
:param filters: dictionary with lists of strings that the urls should contain or not. Keys: positive and negative.
    Urls will be checked to contain at least one positive filter and none of the negatives.
    e.g.
    {positive: [".com",".es"],
    negative: ["facebook","instagram"]}
:param keep_links: (False by default) Wether to keep the found links in each pa

In [16]:
#in standalone
#the filter parameter will make sure that all links contain at least 
# one string of the positive list and none of the negative.
docs = crawler.run(
    query = "https://www.roughguides.com/norway/ ",
    n_articles = 10,
    beam = 5,
    filters = {
        "positive":["norway"],
        "negative":["facebook","instagram"]
    },
    keep_links = False,
    metadata=True,
    summary=True,
    keywords=True,
    path = "articles")

Crawling http://norway.ie?lang=en: 100%|██████████| 10/10 [00:08<00:00,  1.14it/s]                                        


In [ ]:
#example of using the crawler node in a pipeline.

from qdrant_haystack.document_stores import QdrantDocumentStore
from haystack.nodes import EntityExtractor
from haystack.pipelines import Pipeline
from haystack.nodes import PreProcessor

document_store = QdrantDocumentStore(
    ":memory:",
    index="Document",
    embedding_dim=768,
    recreate_index=True,
)

entity_extractor = EntityExtractor(model_name_or_path="dslim/bert-base-NER",flatten_entities_in_meta_data=True)

processor = PreProcessor(
    clean_empty_lines=False,
    clean_whitespace=False,
    clean_header_footer=False,
    split_by="sentence",
    split_length=30,
    split_respect_sentence_boundary=False,
    split_overlap=0
)

indexing_pipeline = Pipeline()
indexing_pipeline.add_node(component=crawler, name="crawler", inputs=['File'])
indexing_pipeline.add_node(component=processor, name="processor", inputs=['crawler'])
indexing_pipeline.add_node(entity_extractor, "EntityExtractor", ["processor"])
indexing_pipeline.add_node(component=document_store, name="document_store", inputs=['EntityExtractor'])

#we can pass the previously seen arguments also
indexing_pipeline.run(query = "https://www.roughguides.com/norway/",
    params={
        "crawler":{
            "n_articles" : 500,
            "beam" : 5,
            "filters" : {
                "positive":["norway"],
                "negative": ["facebook"]
            },
            "keep_links" : False,
            "metadata":True,
            "summary":True,
            "keywords":True,
            "path": "articles"
        }
    })